##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Transfer Learning for the Audio Domain with TensorFlow Lite Model Maker

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/lite/models/modify/model_maker/audio_classification"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/models/modify/model_maker/audio_classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/models/modify/model_maker/audio_classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/tensorflow/tensorflow/lite/g3doc/models/modify/model_maker/audio_classification.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
  <td>
    <a href="https://tfhub.dev/google/yamnet/1"><img src="https://www.tensorflow.org/images/hub_logo_32px.png" />See TF Hub model</a>
  </td>

</table>


In this colab notebook, you'll learn how to use the [TensorFlow Lite Model Maker](https://www.tensorflow.org/lite/models/modify/model_maker) to train a custom audio classification model.

The Model Maker library uses transfer learning to simplify the process of training a TensorFlow Lite model using a custom dataset. Retraining a TensorFlow Lite model with your own custom dataset reduces the amount of training data and time required.

It is part of the [Codelab to Customize an Audio model and deploy on Android](https://codelabs.developers.google.com/codelabs/tflite-audio-classification-custom-model-android).

## Intalling dependencies


In [51]:
!sudo apt -y install libportaudio2
!sudo apt -y install libsndfile1
!pip install tflite-model-maker
!pip install numpy==1.23.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libportaudio2 is already the newest version (19.6.0-1build1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsndfile1 is already the newest version (1.0.28-7ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Import TensorFlow, Model Maker and other libraries

Among the dependencies that are needed, you'll use TensorFlow and Model Maker. Aside those, the others are for audio manipulation, playing and visualizations.

In [52]:
import tensorflow as tf
import tflite_model_maker as mm
from tflite_model_maker import audio_classifier
import os

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import glob
import random

from IPython.display import Audio, Image
from scipy.io import wavfile

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

print(f"TensorFlow Version: {tf.__version__}")
print(f"Model Maker Version: {mm.__version__}")

Mounted at /content/drive
TensorFlow Version: 2.8.4
Model Maker Version: 0.4.2


In [14]:
# @title [Run this] Util functions and data structures.

data_dir = 'drive/MyDrive/COS598D/clips/'
test_files = os.path.abspath(os.path.join(data_dir, '*/*.wav'))

def get_random_audio_file():
  test_list = glob.glob(test_files)
  random_audio_path = random.choice(test_list)
  return random_audio_path

functions and data structures created


In [6]:
random_audio = get_random_audio_file()

## Training the Model

When using Model Maker for audio, you have to start with a model spec. This is the base model that your new model will extract information to learn about the new classes. It also affects how the dataset will be transformed to respect the models spec parameters like: sample rate, number of channels.

[YAMNet](https://tfhub.dev/google/yamnet/1) is an audio event classifier trained on the AudioSet dataset to predict audio events from the AudioSet ontology.

It's input is expected to be at 16kHz and with 1 channel.

You don't need to do any resampling yourself. Model Maker takes care of that for you.

- `frame_length` is to decide how long each traininng sample is. in this caase EXPECTED_WAVEFORM_LENGTH * 3s

- `frame_steps` is to decide how far appart are the training samples. In this case, the ith sample will start at EXPECTED_WAVEFORM_LENGTH * 6s after the (i-1)th sample.

The reason to set these values is to work around some limitation in real world dataset.


In [7]:
spec = audio_classifier.YamNetSpec(
    keep_yamnet_and_custom_heads=True,
    frame_step=3 * audio_classifier.YamNetSpec.EXPECTED_WAVEFORM_LENGTH,
    frame_length=6 * audio_classifier.YamNetSpec.EXPECTED_WAVEFORM_LENGTH)

## Loading the data

Model Maker has the API to load the data from a folder and have it in the expected format for the model spec.

The train and test split are based on the folders. The validation dataset will be created as 20% of the train split.

Note: The `cache=True` is important to make training later faster but it will also require more RAM to hold the data. For the birds dataset that is not a problem since it's only 300MB, but if you use your own data you have to pay attention to it.


In [15]:
train_data = audio_classifier.DataLoader.from_folder(
    spec, os.path.join(data_dir, ''), cache=True)
train_data, validation_data = train_data.split(0.8)
test_data = audio_classifier.DataLoader.from_folder(
    spec, os.path.join(data_dir, ''), cache=True)

## Training the model

the audio_classifier has the [`create`](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/audio_classifier/create) method that creates a model and already start training it. 

You can customize many parameterss, for more information you can read more details in the documentation.

On this first try you'll use all the default configurations and train for 100 epochs.

Note: The first epoch takes longer than all the other ones because it's when the cache is created. After that each epoch takes close to 1 second.

In [16]:
batch_size = 128
epochs = 100

print('Training the model')
model = audio_classifier.create(
    train_data,
    spec,
    validation_data,
    batch_size=batch_size,
    epochs=epochs)

Training the model
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 classification_head (Dense)  (None, 2)                2050      
                                                                 
Total params: 2,050
Trainable params: 2,050
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
2/2 [==============================] - 240s 45s/step - loss: 0.2160 - acc: 0.9493 - val_loss: 0.0302 - val_acc: 1.0000
Epoch 2/100
2/2 [==============================] - 0s 287ms/step - loss: 0.0845 - acc: 0.9928 - val_loss: 0.0135 - val_acc: 1.0000
Epoch 3/100
2/2 [==============================] - 0s 293ms/step - loss: 0.0690 - acc: 0.9855 - val_loss: 0.0093 - val_acc: 1.0000
Epoch 4/100
2/2 [==============================] - 0s 302ms/step - loss: 0.0672 - acc: 0.9928 - val_loss: 0.0076 - val_acc: 1.0000
Epoch 5/100
2/2 [====================

The accuracy looks good but it's important to run the evaluation step on the test data and vefify your model achieved good results on unseed data.

In [17]:
print('Evaluating the model')
model.evaluate(test_data)

Evaluating the model
7/7 [==============================] - 225s 30s/step - loss: 0.0216 - acc: 0.9948


[0.021579409018158913, 0.9948453903198242]

## Understanding your model

When training a classifier, it's useful to see the [confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix). The confusion matrix gives you detailed knowledge of how your classifier is performing on test data.

Model Maker already creates the confusion matrix for you.

In [ ]:
def show_confusion_matrix(confusion, test_labels):
  """Compute confusion matrix and normalize."""
  confusion_normalized = confusion.astype("float") / confusion.sum(axis=1)
  axis_labels = test_labels
  ax = sns.heatmap(
      confusion_normalized, xticklabels=axis_labels, yticklabels=axis_labels,
      cmap='Blues', annot=True, fmt='.2f', square=True)
  plt.title("Confusion matrix")
  plt.ylabel("True label")
  plt.xlabel("Predicted label")

confusion_matrix = model.confusion_matrix(test_data)
show_confusion_matrix(confusion_matrix.numpy(), test_data.index_to_label)

## Testing the model [Optional]

You can try the model on a sample audio from the test dataset just to see the results.

First you get the serving model.

In [ ]:
serving_model = model.create_serving_model()

print(f'Model\'s input shape and type: {serving_model.inputs}')
print(f'Model\'s output shape and type: {serving_model.outputs}')

Coming back to the random audio you loaded earlier

The model created has a fixed input window. 

For a given audio file, you'll have to split it in windows of data of the expected size. The last window might need to be filled with zeros.

In [ ]:
sample_rate, audio_data = wavfile.read(random_audio, 'rb')

audio_data = np.array(audio_data) / tf.int16.max
input_size = serving_model.input_shape[1]

splitted_audio_data = tf.signal.frame(audio_data, input_size, input_size, pad_end=True, pad_value=0)

print(f'Test audio path: {random_audio}')
print(f'Original size of the audio data: {len(audio_data)}')
print(f'Number of windows for inference: {len(splitted_audio_data)}')

You'll loop over all the splitted audio and apply the model for each one of them.

The model you've just trained has 2 outputs: The original YAMNet's output and the one you've just trained. This is important because the real world environment is more complicated than just bird sounds. You can use the YAMNet's output to filter out non relevant audio, for example, on the birds use case, if YAMNet is not classifying Birds or Animals, this might show that the output from your model might have an irrelevant classification.


Below both outpus are printed to make it easier to understand their relation. Most of the mistakes that your model make are when YAMNet's prediction is not related to your domain (eg: birds).

In [ ]:
print(random_audio)

results = []
print('Result of the window ith:  your model class -> score,  (spec class -> score)')
for i, data in enumerate(splitted_audio_data):
  yamnet_output, inference = serving_model(data)
  results.append(inference[0].numpy())
  result_index = tf.argmax(inference[0])
  spec_result_index = tf.argmax(yamnet_output[0])
  t = spec._yamnet_labels()[spec_result_index]
  result_str = f'Result of the window {i}: ' \
  f'\t{test_data.index_to_label[result_index]} -> {inference[0][result_index].numpy():.3f}, ' \
  f'\t({spec._yamnet_labels()[spec_result_index]} -> {yamnet_output[0][spec_result_index]:.3f})'
  print(result_str)


results_np = np.array(results)
mean_results = results_np.mean(axis=0)
result_index = mean_results.argmax()
print(f'Mean result: {test_data.index_to_label[result_index]} -> {mean_results[result_index]}')

## Exporting the model

The last step is exporting your model to be used on embedded devices or on the browser.

The `export` method export both formats for you.

In [18]:
models_path = 'drive/MyDrive/COS598D/tflite_models/'
print(f'Exporing the TFLite model to {models_path}')

model.export(models_path, tflite_filename='whale_tflite_model_1.tflite')

Exporing the TFLite model to drive/MyDrive/COS598D/tflite_models/


You can also export the SavedModel version for serving or using on a Python environment.

In [19]:
model.export(models_path, export_format=[mm.ExportFormat.SAVED_MODEL, mm.ExportFormat.LABEL])

Now we move on to using quantization aware training to further reduce the size of the ML model. We'll save it too.

In [36]:
import tensorflow_model_optimization as tfmot
import pathlib
import time

In [28]:
converter = tf.lite.TFLiteConverter.from_saved_model(models_path+'/saved_model/')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

quant_model_dir = pathlib.Path(models_path+'/saved_model/')/'quantized_model1.tflite'

quant_model_dir.write_bytes(tflite_quant_model)

4041264

Now, read the saved model back as a TFLite model

In [50]:
interpreter = tf.lite.Interpreter(model_path=models_path+"/quantized_model/quantized_model1.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_shape = input_details[0]['shape']
print(input_shape)

i=0
whales_identified=0
whale_samples=0
other_samples_identified=0
other_samples=0
directory_mn='drive/MyDrive/COS598D/clips/mn/'
directory_not_mn='drive/MyDrive/COS598D/clips/not_mn/'

start_time=time.time()
for file in os.listdir(directory_mn):
  if i==1:
    break
  filename = os.fsdecode(file)
  # if this isn't a .wav file, skip
  if filename[-4:] != '.wav':
      continue
  # try opening the file
  waveform, sample_rate = tf.audio.decode_wav(tf.io.read_file(directory_mn+filename))

  waveform = tf.squeeze(tf.expand_dims(waveform,0), [2])  # makes a batch of size 1
  print(input_details[0])
  interpreter.set_tensor(input_details[0]['index'], waveform)
  score = interpreter.invoke()
  if len(score[0]) == 0:
      continue
  score = score[0][0]
  if filename[0:2]=='Mn':
      whale_samples+=1
      if score >= 0.5:
          whales_identified+=1
  else:
      other_samples+=1
      if score < 0.5:
          other_samples_identified+=1
  i+=1
    
for file in os.listdir(directory_not_mn):
  break
  filename = os.fsdecode(file)
  # if this isn't a .wav file, skip
  if filename[-4:] != '.wav':
      continue
      
  # try opening the file
  try:
      waveform, sample_rate = tf.audio.decode_wav(tf.io.read_file(directory_not_mn+filename))
  except:
      print("error encountered while processing "+filename+", continuing with inference")
      continue
  waveform = tf.expand_dims(waveform,0)
  try:
      score = m.predict(waveform)
  except:
      print('could not run inference on ' + filename)
      continue
  if len(score[0]) == 0:
      continue
  score = score[0][0]
  if filename[0:2]=='Mn':
      whale_samples+=1
      if score >= 0.5:
          whales_identified+=1
  else:
      other_samples+=1
      if score < 0.5:
          other_samples_identified+=1
  i+=1
        

end_time = time.time()
print('done processing data, elapsed time: '+str(end_time-start_time))

[    1 15600]
{'name': 'serving_default_audio:0', 'index': 0, 'shape': array([    1, 15600], dtype=int32), 'shape_signature': array([   -1, 15600], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}


ValueError: ignored

## Next Steps

You did it.

Now your new model can be deployed on  mobile devices using [TFLite AudioClassifier Task API](https://www.tensorflow.org/lite/inference_with_metadata/task_library/audio_classifier).

You can also try the same process with your own data with different classes and here is the documentation for [Model Maker for Audio Classification](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/audio_classifier).

Also learn from end-to-end reference apps: [Android](https://github.com/tensorflow/examples/tree/master/lite/examples/sound_classification/android/), [iOS](https://github.com/tensorflow/examples/tree/master/lite/examples/sound_classification/ios).